In [58]:
install.packages("RSQLite",repos="https://cran.r-project.org")
install.packages("dplyr",repos="https://cran.r-project.org")
install.packages("plyr",repos="https://cran.r-project.org")

Warning message:
"package 'RSQLite' is in use and will not be installed"Warning message:
"package 'dplyr' is in use and will not be installed"Warning message:
"package 'plyr' is in use and will not be installed"

In [1]:
library("RSQLite")
library("dplyr")
library("plyr")

Warning message:
"package 'RSQLite' was built under R version 3.3.3"Warning message:
"package 'dplyr' was built under R version 3.3.3"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Warning message:
"package 'plyr' was built under R version 3.3.3"------------------------------------------------------------------------------
You have loaded plyr after dplyr - this is likely to cause problems.
If you need functions from both plyr and dplyr, please load plyr first, then dplyr:
library(plyr); library(dplyr)
------------------------------------------------------------------------------

Attaching package: 'plyr'

The following objects are masked from 'package:dplyr':

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize



In [20]:
db<-src_sqlite("database.sqlite",create=FALSE)#establishing connection with database 
pldf<-as.data.frame(tbl(db,"Player"))
platt<-as.data.frame(tbl(db,"Player_Attributes"))
mtc<-as.data.frame(tbl(db,"Match"))

## Establishing connection to database and extracting a data frame for selecting required variables

In [21]:
player<-data.frame(pldf$player_api_id,pldf$player_name)
names(player)<-c("player_api_id","player_name")

# Calculating the number of appearances and appending it to player dataframe

In [22]:
num_app<-as.data.frame(table(unlist(mtc[1:nrow(mtc),56:77])))[-1,]
names(num_app)<-c("player_api_id","num_appearences")
player<-merge(player,num_app)
head(player)

player_api_id player_name      num_appearences
1 2752          Diego Mainz       81            
2 2768          Jose Dorado       71            
3 2770          Ignacio Gonzalez  16            
4 2790          Alberto Rey       50            
5 2796          Javier Jimenez     1            
6 2802          Pablo Hernandez  144

## Calculating number of leagues played by the player and appending it to player dataframe

In [23]:
arr<-data.frame()
j<-1
leaguelistl<-c(unique(mtc$league_id)[-1],25979)


In [24]:
for(i in leaguelistl){
  array<-as.data.frame(unique(unlist(mtc[j:i,56:77])))
  names(array)<-"val"
  arr<-rbind(arr,array)
  j<-i+1
}


In [25]:
league_table<-(as.data.frame(table(arr)))[-1,]
names(league_table)=c("player_api_id","leagues_played")
player<-merge(player,league_table)
head(player)

player_api_id player_name      num_appearences leagues_played
1 2752          Diego Mainz       81             1             
2 2768          Jose Dorado       71             1             
3 2770          Ignacio Gonzalez  16             2             
4 2790          Alberto Rey       50             1             
5 2796          Javier Jimenez     1             1             
6 2802          Pablo Hernandez  144             3

## Calculating overall rating and overall goalkeeper attributes by summing it and appending to player data frame

In [26]:
playatt<-data.frame(platt$player_api_id,platt$overall_rating)
playatt$gk_overall<-(platt$gk_diving+platt$gk_handling+platt$gk_kicking+platt$gk_positioning+platt$gk_reflexes)
#head(playatt)
names(playatt)<-c("player_api_id","overall_rating","gk_overall")
playatt<-(ddply(playatt,"player_api_id",numcolwise(max)))
player<-merge(player,playatt)
head(arrange(player,desc(gk_overall)))

player_api_id player_name      num_appearences leagues_played overall_rating
1 30717         Gianluigi Buffon 231             1              93            
2 39989         Gregory Coupet    37             2              92            
3 30859         Petr Cech        251             1              89            
4 30657         Iker Casillas    232             2              91            
5 27299         Manuel Neuer     255             1              90            
6 30989         Julio Cesar      202             3              89            
  gk_overall
1 449       
2 447       
3 445       
4 442       
5 440       
6 438

## Writing player dataframe to csv file

In [27]:
write.csv(player,"player.csv")


In [28]:
mtc<-as.data.frame(tbl(db,"Match"))
teamatt<-as.data.frame(tbl(db,"Team_Attributes"))
tm<-as.data.frame(tbl(db,"Team"))

## Calculating buildup play,chance creation and defence attributes by summing it and appending to team dataframe by taking colwise maximum of the obtained dataframe

In [29]:
teamatt[is.na(teamatt)]<-0
teamatt$buildUpPlay<-(teamatt$buildUpPlayDribbling+teamatt$buildUpPlaySpeed+teamatt$buildUpPlayPassing)
teamatt$chanceCreation<-(teamatt$chanceCreationPassing+teamatt$chanceCreationCrossing+teamatt$chanceCreationShooting)
teamatt$defence<-(teamatt$defenceAggression+teamatt$defencePressure+teamatt$defenceTeamWidth)


In [30]:
team<-data.frame(teamatt$team_fifa_api_id,teamatt$team_api_id,teamatt$buildUpPlay,teamatt$chanceCreation,teamatt$defence)
names(team)<-c("fifa_api_id","team_api_id","buildUpPlay","chanceCreation","defence")
team<-(ddply(team,"team_api_id",numcolwise(max)))
head(team)

team_api_id fifa_api_id buildUpPlay chanceCreation defence
1 1601           874      139         186            175    
2 1773        100087      153         157            137    
3 1957        110745      158         163            210    
4 2033        111540      147         165            155    
5 2182           873      176         196            160    
6 2183          1570       80         180            175

In [31]:
teams<-data.frame(tm$team_api_id,tm$team_long_name,tm$team_short_name)
names(teams)=c("team_api_id","team_long_name","team_short_name")

## Getting team shortname and longname and appending it to team dataframe

In [32]:
team<-merge(teams,team)
head(team)

team_api_id team_long_name        team_short_name fifa_api_id buildUpPlay
1 1601        Ruch Chorzów          CHO                874      139        
2 1773        Oud-Heverlee Leuven   O-H             100087      153        
3 1957        Jagiellonia Bialystok BIA             110745      158        
4 2033        S.C. Olhanense        OLH             111540      147        
5 2182        Lech Poznan           POZ                873      176        
6 2183        P. Warszawa           PWA               1570       80        
  chanceCreation defence
1 186            175    
2 157            137    
3 163            210    
4 165            155    
5 196            160    
6 180            175

In [33]:
ht<-data.frame(mtc$home_team_api_id,mtc$home_team_goal)
names(ht)=c("team_api_id","goals_scored")
ht<-(ddply(ht,"team_api_id",numcolwise(sum)))
ht<-arrange(ht,desc(team_api_id))


In [34]:
at<-data.frame(mtc$away_team_api_id,mtc$away_team_goal)
names(at)=c("team_api_id","goals_scored")
at<-(ddply(at,"team_api_id",numcolwise(sum)))
at<-arrange(at,desc(team_api_id))


In [35]:
ht$goals_scored<-ht$goals_scored+at$goals_scored
goal_score<-arrange(ht,desc(goals_scored))

## Calculating number of goals scored from the Match table and appending it to Team dataframe

In [36]:
team<-merge(team,goal_score)
head(team)

team_api_id team_long_name        team_short_name fifa_api_id buildUpPlay
1 1601        Ruch Chorzów          CHO                874      139        
2 1773        Oud-Heverlee Leuven   O-H             100087      153        
3 1957        Jagiellonia Bialystok BIA             110745      158        
4 2033        S.C. Olhanense        OLH             111540      147        
5 2182        Lech Poznan           POZ                873      176        
6 2183        P. Warszawa           PWA               1570       80        
  chanceCreation defence goals_scored
1 186            175     280         
2 157            137     126         
3 163            210     287         
4 165            155     138         
5 196            160     372         
6 180            175     184

In [37]:
write.csv(team,"team.csv")

## Writing team to team.csv